# Environment Set Up

In [1]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

'/workspaces/TumorTracer'

In [2]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

Git folder exists: True


# 3. Model Training

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path                              # Directory where the trained model will be stored
    trained_model_path: Path                    # Path to the trained model
    updated_base_model: Path                    # Path to the updated model with custom layers
    training_data: Path                         # Path to the training dataset
    validation_data: Path                       # Path to the validation dataset
    params_augmentation: bool
    params_image_size: tuple[int, int, int]     # Input image size, e.g., [224, 224, 3]
    params_batch_size: int
    params_epochs: int

In [ ]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH) -> None:
        """
        Reads configuration files (config.yaml and params.yaml), 
        ensures necessary directories exist, and prepares structured config objects.

        Args:
        - config_file_path (str): Path to the config.yaml file.
        - params_file_path (str): Path to the params.yaml file.
        """
        # Validate and load config.yaml
        if not Path(config_file_path).exists():
            logger.error(f"Config file not found at: {config_file_path}")
            raise FileNotFoundError(f"Config file not found at: {config_file_path}")
        self.config = read_yaml(config_file_path)

        # Validate and load params.yaml
        if not Path(config_file_path).exists():
            logger.error(f"Params file not found at: {params_file_path}")
            raise FileNotFoundError(f"Params file not found at: {params_file_path}")
        self.params = read_yaml(params_file_path)

        logger.info(f"Loading configuration from {config_file_path} and parameters from {params_file_path}")

        # Create the root artifacts directory (if not already present)
        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> ModelTrainingConfig:
        """
        Prepares and returns the ModelTrainingConfig object.

        Returns:
        - ModelTrainingConfig: Structured config for training the updated base model.
        """
        training_config = self.config.model_training
        training_params = self.params.model_training

        # Ensure the data_ingestion directory exists
        create_directories([training_config.root_dir])

        training_config = ModelTrainingConfig(
            root_dir=Path(training_config.root_dir),
            trained_model_path=Path(training_config.trained_model_path),
            updated_base_model=Path(training_config.updated_model_path),
            training_data=Path(training_config.training_dataset),
            validation_data=Path(training_config.validation_dataset),
            params_augmentation=training_params.AUGMENTATION,
            params_image_size=tuple(training_params.IMAGE_SIZE),
            params_batch_size=training_params.BATCH_SIZE,
            params_epochs=training_params.EPOCHS,
        )

        logger.info(f"ModelTrainingConfig created with: {training_config}")

        return training_config

In [ ]:
import tensorflow as tf
from cnnClassifier.utils.common import create_directories
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

class ModelTraining:
    def __init__(self, config: ModelTrainingConfig) -> None:
        self.config = config
        self.model = None
    
    
    def get_base_model(self) -> None:
        ...

    
    def training_generator(self) -> None:
        ...


    def train(self) -> None:
        ...


    def _save_model(self, save_path: Path, model: tf.keras.Model) -> None:
        """
        Saves a given model to the specified path.
        """
        try:
            create_directories([save_path.parent])
            model.save(save_path)
            logger.info(f"Model saved at: {save_path}")
        
        except Exception as exception_error:
            logger.error(f"Unexpected error while saving the model at {save_path}: {exception_error}")
            raise 

In [ ]:
try:
    config_manager = ConfigurationManager()
    training_config = config_manager.get_training_config()

    training_constructor = ModelTraining(config=training_config)
    training_constructor.get_base_model()
    training_constructor.training_generator()
    training_constructor.train()

except Exception as exception:
    logger.exception(f"Unexpected error during model training pipeline: {exception}")
    raise exception